# Improving movie recommendations for movie snobs - I. Wrangling

## Introduction

This project examines how to improve the precision of recommender systems for movie snobs. This section covers an exploratory analysis of the ratings dataset.

Section I covers the following tasks:
1. Extracting the basic ratings data and canonical (1,001 Movies...) list. 
2. Wrangling (including inserting the canonical list into the movie ratings dataset).
3. Joining the Grouplens tables together

### 1. Initial summary of ratings and canonical data

There are five datasets of relevance on the Grouplens website.

| Name | Movies | Ratings | Userdata | Release date |
| --- | --- | --- | --- | --- |
| New research | 27K | 20M | None | 2016 |
| Education (small) | 9K | 100K | None | 2018 |
| Education (large) | 58K | 27M | None | 2018 |
| Older (100K) | 1.7K | 100K| age, gender, occupation, zip | 1998 |
| Older (1M) | 4K  | 1M | age, gender, occupation, zip | 2003 |

The latter two are useful for identifying clusters of people but are a bit small, while the first and third are best if trying to identify features of movies themselves. Given the obscurity of some of the canonical movies, we chose the largest dataset, Education (large).

The <a href='https://1001films.fandom.com/wiki/The_List'> canonical list</a> from "1,001 Movies..." had 1,222 entries due to additions and deletions across the different editions..


In [1]:
# Basic packagesused
import pandas as pd
import numpy as np


In [2]:
# Downloading the Grouplens data (in this case just one file, but we did all of them as a list)

#import requests, zipfile, io
# Importing the ratings data
# First download and extract the files (there's a bunch so use a list and loop)
#list_of_urls = ['http://files.grouplens.org/datasets/movielens/ml-latest.zip']
#for url in list_of_urls:
#    ratings_small_file = requests.get(url)
#    z = zipfile.ZipFile(io.BytesIO(ratings_small_file.content))
#    z.extractall()

In [3]:
import requests
from bs4 import BeautifulSoup
# Importing the 1,001 list and converting it to a list
snob_url = 'https://1001films.fandom.com/wiki/The_List'
snob_text= requests.get(snob_url)
soup = BeautifulSoup(snob_text.content, 'html.parser')
basic_list = (soup.body.find_all('b'))
thousand_list = [item.text for item in basic_list]

In [4]:
# Here I get all three files into dataframes
# first, read the correct downloaded csvs (from separate subfolders)
ed_large_movies = pd.read_csv('ml-latest/movies.csv', sep = ',', header = 0)

# The two older files have a different compression structure and require a different technique
#older_small_movies = pd.read_csv('ml-100k/movies.csv', sep = ',', header = 0)
#older_large_movies = pd.read_csv('ml-1m/movies.csv', sep = ',', header = 0)

# Converting the 1,001 list to a dataframe and dropping the header row
thousandone_movies = pd.DataFrame(thousand_list, columns = ['title']).drop(0)

# The following code can be modified to check they are all dataframes
#print(thousandone_movies.head())
#print(ed_large_movies.head())
#thousandone_movies.info()


### 2. Wrangling (including inserting the canonical list into the ratings data)

#### 2a. Wrangling the canonical list.
The canonical list came as a set of strings, so we needed to extract the year of release from those strings and then find a way to match the titles.

In [5]:
# Create columns of movie years in each database
# Make sure the titles don't have trailing spaces
thousandone_movies['title'] = thousandone_movies['title'].str.rstrip()
ed_large_movies['title'] = ed_large_movies['title'].str.rstrip()
# Then take the slices (the years are in parantheses at the end of the title)
thousandone_movies['year'] = [title[slice(-5,-1)] for title in thousandone_movies['title']]
ed_large_movies['year'] = [title[slice(-5,-1)] for title in ed_large_movies['title']]

# Then convert these strings to numbers (there is one title missing a year!)
# Define a conversion function
def ConvertYear(value):
    '''This function converts integer strings to integers and non-integer strings to zero'''
    try:
        return int(value)
    except: 
        return 0
# Then apply it to the columns for both thousandone_movies and ed_choices
thousandone_movies['year'] = thousandone_movies['year'].apply(lambda year: ConvertYear(
    year))
ed_large_movies['year'] = ed_large_movies['year'].apply(lambda year: ConvertYear(year))


#### 2b. Inserting the canonical list into the ratings data.
No doubt the hardest wrangling job was adjusting for any naming discrepancies in titles between files. I tried straight matching first joined the movie sets to the 1,001 list and the match rate was 49%. Then I tried some simple string manipulations and brought it up to 72%. I then used the nltk package's tokenizer, and an 80% match rate (974 out of 1,222).

fuzzywuzzy, a string matching library, compares a string to a set of targets and returns the best match. The basic extractOne method led to the highest match rate yet (91.6%), but 103 mismatches remained.

In [ ]:
# This cell does the first version of the fuzzywuzzy matching using the default sorter
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Select the list of titles
choices_ed = ed_large_movies['title']

# Test it on a small part
#test_titles = thousandone_movies.iloc[0:30]

def Matcher(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices)
    return title_match

# original version (103 mismatches)
thousandone_movies['return_match'] = thousandone_movies['title'].apply(lambda title: Matcher(
    title, choices_ed))

Matching using fuzzywuzzy's in-built tokenizer led to only 48 mismatches.
I then inspected the mismatches and noticed that most were to movies with similar titles but very different years (e.g. October/Oktyabr (1929) matched to October Sky (1999)). As such, I thought to limit the number of targets to ±1 year from the known year of release (exact year would not work as sometimes the databases differed in their year of release). Having a reduced matching list also greatly sped up the matching algorithm. To do this, I filtered the matching list

In [ ]:
# The matching done via fuzzywuzzy's tokenizer sorting algorithm.
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Get the list of possible matches

choices_ed = ed_large_movies['title']

# Define the matching function correcly
def Matcher(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices)
    return title_match

def Matcher_token(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices, 
                                                            scorer=fuzz.token_sort_ratio)
    return title_match

# Call it on all the rows we wish to have matched via a lambda function
thousandone_movies['return_match'] = thousandone_movies['title'].apply(lambda title: Matcher_token
                                                                       (title, choices_ed))
# Save the file for checking
thousandone_movies.to_csv('MatcherWithFuzzyWuzzyToken.csv')

<b> Here's the final version of the matching </b>

In [6]:
%%time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# Specify the matching function (we only need one of the outputs)
def Matcher(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices)
    return title_match
# And here's a function for using the tokenizer
def Matcher_token(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices, 
                                                            scorer=fuzz.token_sort_ratio)
    return title_match

#Define a filter to return targets for +/-1 year only
def YearFilter (year):
    years = [year-1, year, year+1]
    return ed_large_movies[ed_large_movies.year.isin(years)].title

# Running the tokenizer over the filtered target set
for index, row in thousandone_movies.iterrows():
    # call the filter
    targets = YearFilter(row.year)
    # update the new cell work out the matcher
    thousandone_movies.loc[index,'return_match'] = Matcher_token(row.title, targets)
thousandone_movies.to_csv('thousandone_forlooped_matcher.csv')

# I then had to visually inspect the forlooped_matcher file to calculate the number of mismatches (38/1224)

CPU times: user 46.3 s, sys: 538 ms, total: 46.8 s
Wall time: 54.3 s


(You can ignore the cell below, it was for testing purposes)

In [24]:
''' This cell contains a test version of the matching routine'''
#Testing one title
test_title = thousandone_movies.iloc[1:2]
#test_title = thousandone_movies.iloc[1:2].drop(['return_match'], axis=1)
#print(test_title)
#Testing two titles
test_titles = thousandone_movies.iloc[1:4]
#test_titles = thousandone_movies.iloc[1:3].drop(['return_match'], axis=1)
#print(test_title)

#def ChoiceList(array_of_targets):
#    return YearFilter(array_of_targets.year).title

Train_Targets = YearFilter(test_title.year)
#print(Train_Targets)
#x = Matcher('The Melo', Train_Targets)

# Running the basic function over the whole database
for index, row in test_titles.iterrows():
    # call the filter
    targets = YearFilter(row.year)
#    print(targets)
    # update the new cell work out the matcher
    test_titles.loc[index,'return_match'] = Matcher_token(row.title, targets)
    
# This one works
#test_titles['return_match'] = test_titles['title'].apply(lambda x: 
#                                                       Matcher_token(x, YearFilter(test_title.year)))

print(test_titles)

#Great Train Robbery <- gets the right comp (1902-1904)
#Birth of a nation <- uses the comparison set from the first row (1902-1904)

                            title  year                    return_match
2  The Great Train Robbery (1903)  1903  The Great Train Robbery (1903)
3    The Birth of a Nation (1915)  1915   Birth of a Nation, The (1915)
4             Les Vampires (1915)  1915            Vampires, Les (1915)


/Users/jonathangerber/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jonathangerber/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


The films were now matched in the 1,001 movie file but this file needed to be joined to the Grouplens data file.

In [7]:
# Add the indicator variable to the canonical list.
thousandone_movies['canonical'] = 1
#print(thousandone_movies.head())

# Add the canonical indicator to the movie file, drop the irrelevant columns 
#and fill the missing values with zeroes
ed_large_movies = pd.merge(ed_large_movies, thousandone_movies, left_on='title', right_on='return_match', how='outer', 
         suffixes=('', '_canon')).drop(['year_canon', 
                                        'return_match', 'title_canon'], axis=1).fillna({'canonical':0})

print(ed_large_movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  canonical  
0  Adventure|Animation|Children|Comedy|Fantasy  1995        1.0  
1                   Adventure|Children|Fantasy  1995        0.0  
2                               Comedy|Romance  1995        0.0  
3                         Comedy|Drama|Romance  1995        0.0  
4                                       Comedy  1995        0.0  


#### 2c. Updating the release date
I used an API wrapper to replace the year of release with date of release.

In [10]:
# Insert the correct tmdb movie IDs into the Grouplens file
tmdb_links = pd.read_csv('ml-latest/links.csv', sep=',', header=0)

ed_large_movies = pd.merge(ed_large_movies, tmdb_links, left_on='movieId', right_on='movieId', how='outer', 
         suffixes=('', 'tmdb')).drop(['imdbId'], axis=1)

# Use the tmdb IDs to get the correct release dates via the tmdbsimple API library
import tmdbsimple as tmdb
tmdb.API_KEY = 'b70779efd983d6156620d110a8d3b3c4'

# Also need to limit the rate for the call
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=7, period=2)
def ReleaseDate(tmdb_ID):
    try: 
        movie = tmdb.Movies(tmdb_ID)
        response = movie.info()
        return movie.release_date
    except:
        return np.nan
%time

# Here I make a looping function to check that it works

ed_large_movies['release_date'] = np.NaN
i = 0
batchsize = 100
calls = ed_large_movies.shape[0] // batchsize

for batch in range(0,calls):
    j = i + 100
    ed_large_movies.ix[i:j,'release_date'] = ed_large_movies.ix[i:j,'tmdbId'].apply(
    lambda x: ReleaseDate(x))
    i+=batchsize
    print('Batch {:.0f} is processed'.format(batch))

#for batch in range(0,batchlist):
    
#for 
#ed_large_movies.ix[0:10,'release_date'] = ed_large_movies.ix[0:10,'tmdbId'].apply(
#    lambda x: ReleaseDate(x))

#print(ed_large_movies.head())

#ed_large_movies['release_date'] = ed_large_movies['tmdbId'].apply(lambda x:ReleaseDate(x))
#canonical_ed = ed_large_movies[ed_large_movies['canonical']==1]
#canonical_ed['release_date'] = canonical_ed['tmdbId'].apply(lambda x: ReleaseDate(x))


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


/Users/jonathangerber/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/jonathangerber/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Batch 0 is processed
Batch 1 is processed
Batch 2 is processed
Batch 3 is processed
Batch 4 is processed
Batch 5 is processed
Batch 6 is processed
Batch 7 is processed
Batch 8 is processed
Batch 9 is processed
Batch 10 is processed
Batch 11 is processed
Batch 12 is processed
Batch 13 is processed
Batch 14 is processed
Batch 15 is processed
Batch 16 is processed
Batch 17 is processed
Batch 18 is processed
Batch 19 is processed
Batch 20 is processed
Batch 21 is processed
Batch 22 is processed
Batch 23 is processed
Batch 24 is processed
Batch 25 is processed
Batch 26 is processed
Batch 27 is processed
Batch 28 is processed
Batch 29 is processed
Batch 30 is processed
Batch 31 is processed
Batch 32 is processed
Batch 33 is processed
Batch 34 is processed
Batch 35 is processed
Batch 36 is processed
Batch 37 is processed
Batch 38 is processed
Batch 39 is processed
Batch 40 is processed
Batch 41 is processed
Batch 42 is processed
Batch 43 is processed
Batch 44 is processed
Batch 45 is processe

Batch 361 is processed
Batch 362 is processed
Batch 363 is processed
Batch 364 is processed
Batch 365 is processed
Batch 366 is processed
Batch 367 is processed
Batch 368 is processed
Batch 369 is processed
Batch 370 is processed
Batch 371 is processed
Batch 372 is processed
Batch 373 is processed
Batch 374 is processed
Batch 375 is processed
Batch 376 is processed
Batch 377 is processed
Batch 378 is processed
Batch 379 is processed
Batch 380 is processed
Batch 381 is processed
Batch 382 is processed
Batch 383 is processed
Batch 384 is processed
Batch 385 is processed
Batch 386 is processed
Batch 387 is processed
Batch 388 is processed
Batch 389 is processed
Batch 390 is processed
Batch 391 is processed
Batch 392 is processed
Batch 393 is processed
Batch 394 is processed
Batch 395 is processed
Batch 396 is processed
Batch 397 is processed
Batch 398 is processed
Batch 399 is processed
Batch 400 is processed
Batch 401 is processed
Batch 402 is processed
Batch 403 is processed
Batch 404 i

In [ ]:
#print(ed_large_movies.tail())

# write the resulting file for later use
ed_large_movies.to_csv('RightDates.csv')

In [14]:
# read the file again
ed_large_movies = pd.read_csv('RightDates.csv')

# Remove some duplicate columns
ed_large_movies = ed_large_movies.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'tmdbIdtmdb'])

# Check the file
print(ed_large_movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  canonical   tmdbId  \
0  Adventure|Animation|Children|Comedy|Fantasy  1995        1.0    862.0   
1                   Adventure|Children|Fantasy  1995        0.0   8844.0   
2                               Comedy|Romance  1995        0.0  15602.0   
3                         Comedy|Drama|Romance  1995        0.0  31357.0   
4                                       Comedy  1995        0.0  11862.0   

  release_date  
0   1995-10-30  
1   1995-12-15  
2   1995-12-22  
3   1995-12-22  
4   1995-12-08  


### 3. Joining the grouplens tables

#### 3.1 Loading other files
The Grouplens sets have several linked tables. 
The 'tags' file contains a list of tags derived from reviews and user tags. It also gives the date when each tag was given.

In [16]:
tags = pd.read_csv('ml-latest/tags.csv', sep = ',', header = 0)
print(tags.head())
print(tags.shape)

from datetime import datetime
tags['date'] = tags['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
np.min(tags['date'])

print(tags.head())

   userId  movieId           tag   timestamp
0      14      110          epic  1443148538
1      14      110      Medieval  1443148532
2      14      260        sci-fi  1442169410
3      14      260  space action  1442169421
4      14      318  imdb top 250  1442615195
(1108997, 4)
   userId  movieId           tag   timestamp                date
0      14      110          epic  1443148538 2015-09-24 22:35:38
1      14      110      Medieval  1443148532 2015-09-24 22:35:32
2      14      260        sci-fi  1442169410 2015-09-13 14:36:50
3      14      260  space action  1442169421 2015-09-13 14:37:01
4      14      318  imdb top 250  1442615195 2015-09-18 18:26:35


The 'ratings' file contains a list of ratings by user and date.

In [17]:
ratings = pd.read_csv('ml-latest/ratings.csv', sep = ',', header = 0)
ratings['date'] = ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
print('The earliest movie rating was made on {}'.format(np.min(ratings['date'])))
print('The latest movie rating was made on {}'.format(np.max(ratings['date'])))
print(ratings.shape)
print(ratings.head())

The earliest movie rating was made on 1995-01-09 06:46:44
The latest movie rating was made on 2018-09-26 02:59:09
(27753444, 5)
   userId  movieId  rating   timestamp                date
0       1      307     3.5  1256677221 2009-10-27 17:00:21
1       1      481     3.5  1256677456 2009-10-27 17:04:16
2       1     1091     1.5  1256677471 2009-10-27 17:04:31
3       1     1257     4.5  1256677460 2009-10-27 17:04:20
4       1     1449     4.5  1256677264 2009-10-27 17:01:04


The 'genome-score' file contains loadings of the tag set to each movie (see Vig et al reference). This genome appears to be built from a relatively small sample from an academic paper. The 'genome-tag' file contains identifying information for each tag. The 'links' file contains identifying information for each movie title.

In [21]:
# Briefly looking at last three files
genome_scores = pd.read_csv('ml-latest/genome-scores.csv', sep = ',', header = 0)
print('Genome score header')
print(genome_scores.head())
genome_tags = pd.read_csv('ml-latest/genome-tags.csv', sep = ',', header = 0)
print('Genome tags header')
print(genome_tags.head())
tmdb_links = pd.read_csv('ml-latest/links.csv', sep=',', header=0)
print('Link header')
print(tmdb_links.head())

Genome score header
   movieId  tagId  relevance
0        1      1    0.02900
1        1      2    0.02375
2        1      3    0.05425
3        1      4    0.06875
4        1      5    0.16000
Genome tags header
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s
Link header
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


#### 3.3 Joining all the files.
The current analysis needs only one file, a long-format file of ratings with all the necessary identifying movie data (including canonical status and date of release).

In [53]:
# Relevant join to get the file we need.

# First doublecheck the file structure
#print(ratings.head())
#print(ed_large_movies.head())

# Here's the join (have to drop duplicates in the ed_large list or else some ratings are counted twice)
rating2 = pd.merge(ratings, ed_large_movies.drop_duplicates(subset=['movieId']), how='left', on='movieId').drop(
    ['title','genres'], axis=1)  # we also dropped the long string columns

# A simple check to make sure we didn't get any extra lines
print(ratings.shape[0]==rating2.shape[0])

# Then save that dataframe for later use


True


Saving the full file for later use

In [ ]:
# We store the file for use in later sections of this report
rating2.to_pickle('Pickled_df')

The data file we will use is now cleaned and wrangled.

References:
Jesse Vig, Shilad Sen, and John Riedl. 2012. The Tag Genome: Encoding Community Knowledge to Support Novel Interaction. ACM Trans. Interact. Intell. Syst. 2, 3: 13:1–13:44. https://doi.org/10.1145/2362394.2362395